In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
import pandas as pd
import matplotlib.pyplot as plt
from snowflake.cortex import complete

# Get data from Snowflake
session = get_active_session()
query = """
SELECT
    *
FROM
    REVIEWS_WITH_SENTIMENT
"""
df_reviews = session.sql(query).to_pandas()

# Convert date columns to datetime
df_reviews['REVIEW_DATE'] = pd.to_datetime(df_reviews['REVIEW_DATE'])
df_reviews['SHIPPING_DATE'] = pd.to_datetime(df_reviews['SHIPPING_DATE'])

df_string = df_reviews.to_string(index=False)

def create_avalanche_prompt(user_question: str, dataframe_context: str) -> str:
    """Creates the prompt for the LLM."""
    prompt = f"""
You are a helpful AI chat assistant. Answer the user's question based on the provided
context data from customer reviews provided below.

Use the data in the <context> section to inform your answer about customer reviews or sentiments
if the question relates to it. If the question is general and not answerable from the context, answer naturally. Do not explicitly mention "based on the context" unless necessary for clarity.

<context>
{dataframe_context}
</context>

<question>
{user_question}
</question>
"""
    return prompt

In [ ]:
# Initialize the Streamlit app
st.title("Avalanche Streamlit App")

# Visualization: Average Sentiment by Product
st.subheader("Average Sentiment by Product")
product_sentiment = df_reviews.groupby("PRODUCT")["SENTIMENT_SCORE"].mean().sort_values()

fig, ax = plt.subplots()
product_sentiment.plot(kind="barh", ax=ax, title="Average Sentiment by Product")
ax.set_xlabel("Sentiment Score")
plt.tight_layout()
st.pyplot(fig)

# Product filter on the main page
st.subheader("Filter by Product")
products = df_reviews['PRODUCT'].unique()
selected_products = st.multiselect(
    "Select Products to View:",
    options=products,
    default=products  # Default to showing all products
)

# Filter data based on selected products
filtered_data = df_reviews[df_reviews['PRODUCT'].isin(selected_products)]

# Display the filtered data as a table
st.subheader("Filtered Data Table")
st.dataframe(filtered_data)

# Visualization: Sentiment Distribution for Selected Products
st.subheader("Sentiment Distribution for Selected Products")
fig, ax = plt.subplots()
filtered_data['SENTIMENT_SCORE'].hist(ax=ax, bins=20)
ax.set_title("Distribution of Sentiment Scores")
ax.set_xlabel("Sentiment Score")
ax.set_ylabel("Frequency")
st.pyplot(fig)

# Chatbot for Q&A
st.subheader("Ask Questions About Your Data")
user_question = st.text_input("Enter your question here:")

if user_question:
    response = complete(model="claude-3-5-sonnet", prompt=create_avalanche_prompt(user_question, df_string), session=session)
    st.write(response)